#Olá, bem-vindo(a) ao meu Chatbot sobre a LGPD. Caso queira reutilizar o código basta incluir em GOOGLE_API_KEY a sua própria chave do Google AI STUDIO. Não se esqueça de deixar sua chave entre as aspas.

OBS: O arquivo lgpd_doc.json encontra-se disponível para download na página inicial deste projeto. Desta forma, você poderá treinar seu próprio modelo com o models/embedding-001

# Importação de Bibliotecas e Configuração da API

In [72]:
!pip install -q -U google-generativeai # -q -U (tornar o código menos verboso)

import google.generativeai as genai
import pandas as pd
import json
import numpy as np

# API Key protegida com Secrets do Google Colab
GOOGLE_API_KEY = "insira_aqui_a_sua_chave"
genai.configure(api_key=GOOGLE_API_KEY)

# Carregamento e Processamento dos Dados

In [73]:
# Carregamento do arquivo JSON com os documentos sobre LGPD
df = pd.read_json("lgpd_doc.json")

# Função para gerar embeddings dos documentos
def embed_fn(titulo, conteudo):
    embedding = genai.embed_content(
        model="models/embedding-001",
        content=f"{titulo} {conteudo}",  # Combina título e conteúdo para o embedding
        task_type="RETRIEVAL_DOCUMENT",
    )["embedding"]
    return embedding

# Criação de embeddings para todos os documentos do DataFrame
df["embeddings"] = df.apply(lambda row: embed_fn(row["titulo"], row["conteudo"]), axis=1)


In [74]:
df.head()

,titulo,conteudo,embeddings
0,Introdução à LGPD,"A Lei Geral de Proteção de Dados (LGPD), Lei n...","[0.016463513, 0.01615148, -0.044074014, 0.0363..."
1,Princípios Fundamentais,A LGPD é baseada em princípios essenciais que ...,"[0.03152443, 0.013138074, -0.04572051, 0.03739..."
2,Direitos dos Titulares,A LGPD assegura aos titulares de dados uma sér...,"[0.003222508, 0.02588983, -0.05301235, 0.03621..."
3,Obrigações dos Agentes de Tratamento,A LGPD impõe uma série de obrigações aos contr...,"[0.029915368, 0.0048636394, -0.03058556, 0.039..."
4,Sanções,O descumprimento da LGPD pode resultar em sanç...,"[0.02137964, -0.008557823, -0.060764566, 0.031..."


#  Função de Busca por Relevância

In [75]:
# Função para encontrar o documento mais similar à pergunta do usuário
def gerar_buscar_consulta(consulta, dataframe, model):
    # Gera o embedding da pergunta
    embedding_consulta = genai.embed_content(
        model=model, content=consulta, task_type="RETRIEVAL_QUERY"
    )["embedding"]

    # Calcula o produto escalar entre o embedding da pergunta e os embeddings dos documentos
    produtos_escalares = np.dot(np.stack(df["embeddings"]), embedding_consulta)

    # Encontra o índice do documento com maior similaridade (maior produto escalar)
    indice = np.argmax(produtos_escalares)

    # Retorna o conteúdo do documento mais relevante
    return df.iloc[indice]["conteudo"]

# Configuração do Gemini Pro

In [ ]:
model_gemini = genai.GenerativeModel(model_name="gemini-pro")
print("Olá! Sou seu assistente jurídico.\n")

while True:
    prompt = input("O que você quer saber sobre a LGPD? ")

    if prompt.lower() == "fim":
        break

    trecho_relevante = gerar_buscar_consulta(prompt, df, "models/embedding-001")

    resposta = model_gemini.generate_content(
        f"""Considerando o seguinte trecho da LGPD: '{trecho_relevante}',
        forneça uma resposta clara, concisa e informativa para a seguinte pergunta: '{prompt}'.
        Utilize linguagem jurídica adequada para um advogado."""
    )
    resposta_texto = resposta.text
    print(resposta_texto + "\n")

Olá! Sou seu assistente jurídico.

O que você quer saber sobre a LGPD? O que é a LGPD?
A Lei Geral de Proteção de Dados (LGPD), Lei nº 13.709/2018, é uma legislação que regulamenta a coleta, o tratamento e a proteção de dados pessoais no Brasil, promovendo a proteção aos direitos fundamentais dos titulares e o desenvolvimento econômico e social do país.

